In [ ]:
import os
import torch
import transformers
import pandas as pd
from transformers.trainer import *
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaTokenizerFast
from torch.utils.data import SequentialSampler
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, RobertaModel
from transformers import DataCollatorForLanguageModeling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6"

In [ ]:
def load_data(data_directory_path):
  all_data = []
  total_sentence = 0
  for filename in os.listdir(data_directory_path):
    file_path = os.path.join(data_directory_path, filename)
    if os.path.isfile(file_path):
      print(f'Reading file: {filename}')
      with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
          total_sentence+=1
          all_data.append(line.strip())
  print(f"total sentece in directory: {total_sentence}")
  return all_data

In [ ]:
train_directory_path = '/content/drive/MyDrive/VU Thesis/Code/Data/babylm_10M'
sequences = load_data(train_directory_path)
print('='*50)

dev_directory_path = '/content/drive/MyDrive/VU Thesis/Code/Data/babylm_dev'
devset = load_data(dev_directory_path)
print('='*50)

test_directory_path = '/content/drive/MyDrive/VU Thesis/Code/Data/babylm_test'
test_data = load_data(test_directory_path)
print('='*50)

print(len(sequences))
print(len(devset))
print(len(test_data))

Reading file: wikipedia.train
Reading file: gutenberg.train
Reading file: open_subtitles.train
Reading file: simple_wikipedia.train
Reading file: qed.train
Reading file: aochildes.train
Reading file: bnc_spoken.train
Reading file: switchboard.train
Reading file: children_stories.train
Reading file: cbt.train
total sentece in directory: 1058740
Reading file: aochildes.dev
Reading file: qed.dev
Reading file: open_subtitles.dev
Reading file: children_stories.dev
Reading file: cbt.dev
Reading file: bnc_spoken.dev
Reading file: gutenberg.dev
Reading file: simple_wikipedia.dev
Reading file: wikipedia.dev
Reading file: switchboard.dev
total sentece in directory: 1026747
Reading file: aochildes.test
Reading file: gutenberg.test
Reading file: children_stories.test
Reading file: cbt.test
Reading file: bnc_spoken.test
Reading file: wikipedia.test
Reading file: switchboard.test
Reading file: simple_wikipedia.test
Reading file: qed.test
Reading file: open_subtitles.test
total sentece in directory: 

In [ ]:
tokenizer_folder = '/content/drive/MyDrive/VU Thesis/Code/baby_models/tokenizer_V40_folder'

# if not os.path.exists(tokenizer_folder):
  # os.mkdir(tokenizer_folder)

In [ ]:
def train_tokenizer(sequence_data, vocab_size, tokenizer_folder):
  # Initialize tokenizer
  tokenizer = ByteLevelBPETokenizer()

  tokenizer.train_from_iterator(sequences,
                                vocab_size=vocab_size,
                                min_frequency=2,
                                show_progress=True,
                                special_tokens=[
                                    "<s>",
                                    "<pad>",
                                    "</s>",
                                    "<unk>",
                                    "<mask>"]
                                )
  # Save tokenizer
  tokenizer.save_model(tokenizer_folder)

if not os.path.exists(tokenizer_folder):
  os.mkdir(tokenizer_folder)
  train_tokenizer(sequences, 40_960, tokenizer_folder)

In [ ]:
# create CustomDataset class
class CustomDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.examples = []
    self.mask = []
    max_length = 512
    for example in data:
      x=tokenizer.encode_plus(example, max_length = max_length, truncation=True, padding=True)
      self.examples += [x.input_ids]
      self.mask += [x.attention_mask]

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, i):
    return torch.tensor(self.examples[i])

In [ ]:
def get_train_dataloader(self) -> DataLoader:
    """
    Returns the training :class:`~torch.utils.data.DataLoader`.

    Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
    to distributed training if necessary) otherwise.

    Subclass and override this method if you want to inject some custom behavior.
    """
    if self.train_dataset is None:
        raise ValueError("Trainer: training requires a train_dataset.")

    return DataLoader(
        self.train_dataset,
        batch_size=self.args.train_batch_size,
        sampler=SequentialSampler(self.train_dataset),
        collate_fn=self.data_collator,
        drop_last=self.args.dataloader_drop_last,
        num_workers=self.args.dataloader_num_workers,
        shuffle=False
    )

In [ ]:
def train_baby_lm(babaylm_config, tokenizer_path, train_data, dev_data, epochs_start, epochs_number, batch_size):
    model = RobertaForMaskedLM(config=babaylm_config)
    print('Model parameters: ',model.num_parameters())
    max_length = 512
    tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path, max_len=max_length)

    # Define data collator
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
    Trainer.get_train_dataloader = get_train_dataloader

    for i in range(epochs_start, epochs_start + epochs_number):

        if os.path.exists(f'{model_path}/model_e{i}_v40_l6'):
            model = RobertaForMaskedLM.from_pretrained(f'{model_path}/model_e{i}_v40_l6')
            print(f"pretrained model load (from Epoch {i})...")

        else:
            print("No checkpoint ... Create New one")
            model = RobertaForMaskedLM(config=babaylm_config)
            print('Model parameters: ',model.num_parameters())

        # batch_size = 16
        # Define training arguments
        training_args = TrainingArguments(
            output_dir=f'{model_path}/run_model_{i+1}_folder',
            overwrite_output_dir=True,
            eval_strategy = 'epoch',
            num_train_epochs=1,
            learning_rate=1e-4,
            weight_decay=0.01,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            save_strategy="epoch",
            # save_steps=8192,
            save_total_limit=1,
            max_steps=int(len(train_dataset)/ batch_size)
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_data,#train_dataset,
            eval_dataset=dev_data,#eval_dataset,
            tokenizer=tokenizer)

        trainer.train()
        trainer.save_model(f'{model_path}/model_e{i+1}_v40_l6')

In [ ]:
import pickle
import os

max_length = 512
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=max_length)

train_custom_data_path =  "/content/drive/MyDrive/VU Thesis/Code/baby_models/train_V40.pkl"
dev_custom_data_path = "/content/drive/MyDrive/VU Thesis/Code/baby_models/dev_V40.pkl"

if os.path.isfile(train_custom_data_path):
  with open(train_custom_data_path, 'rb') as file:
    train_dataset = pickle.load(file)
else:
  train_dataset = CustomDataset(sequences, tokenizer)
  with open(train_custom_data_path, 'wb') as file:
    pickle.dump(train_dataset, file)


if os.path.isfile(dev_custom_data_path):
  with open(dev_custom_data_path, 'rb') as file:
    eval_dataset = pickle.load(file)
else:
  eval_dataset = CustomDataset(devset, tokenizer)
  with open(dev_custom_data_path, 'wb') as file:
    pickle.dump(eval_dataset, file)

In [ ]:
# Epochs 1-4
config = RobertaConfig(
    vocab_size=40960,
    num_hidden_layers=6,
    max_position_embeddings=514,
    num_attention_heads=8,
    type_vocab_size=1,
    intermediate_size=3072,
    layer_norm_eps = 1e-05,
    hidden_size=256,
    initializer_range=0.02,
    classifier_dropout = None,
    hidden_act="gelu",
    position_embedding_type= "absolute",
    hidden_dropout_prob= 0.1,
    attention_probs_dropout_prob= 0.1,
)

epochs_start = 0
epochs = 5
batch_size = 32

train_baby_lm(config, tokenizer_folder, train_dataset, eval_dataset, epochs_start, epochs, batch_size)

Model parameters:  21767680
No checkpoint ... Create New one
Model parameters:  21767680


<ipython-input-10-6deffa7b05d7>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


...

<ipython-input-10-6deffa7b05d7>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
0,5.366800,4.348996


pretrained model load (from Epoch 4)...


<ipython-input-10-6deffa7b05d7>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


In [ ]:
# Epoch 5
config = RobertaConfig(
    vocab_size=40960,
    num_hidden_layers=6,
    max_position_embeddings=514,
    num_attention_heads=8,
    type_vocab_size=1,
    intermediate_size=3072,
    layer_norm_eps = 1e-05,
    hidden_size=256,
    initializer_range=0.02,
    classifier_dropout = None,
    hidden_act="gelu",
    position_embedding_type= "absolute",
    hidden_dropout_prob= 0.1,
    attention_probs_dropout_prob= 0.1,
)

epochs_start = 4
epochs = 1
batch_size = 32

train_baby_lm(config, tokenizer_folder, train_dataset, eval_dataset, epochs_start, epochs, batch_size)

Model parameters:  21767680
pretrained model load (from Epoch 4)...


<ipython-input-10-6deffa7b05d7>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: farnaz-banifatemi-vu (farnaz-banifatemi-vu-vrije-universiteit-amsterdam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch,Training Loss,Validation Loss
0,5.370600,4.308784


In [ ]:
# Epochs 6-10
config = RobertaConfig(
    vocab_size=40960,
    num_hidden_layers=6,
    max_position_embeddings=514,
    num_attention_heads=8,
    type_vocab_size=1,
    intermediate_size=3072,
    layer_norm_eps = 1e-05,
    hidden_size=256,
    initializer_range=0.02,
    classifier_dropout = None,
    hidden_act="gelu",
    position_embedding_type= "absolute",
    hidden_dropout_prob= 0.1,
    attention_probs_dropout_prob= 0.1,
)

epochs_start = 5
epochs = 5
batch_size = 32

train_baby_lm(config, tokenizer_folder, train_dataset, eval_dataset, epochs_start, epochs, batch_size)

Model parameters:  21767680
pretrained model load (from Epoch 5)...


<ipython-input-10-6deffa7b05d7>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: farnaz-banifatemi-vu (farnaz-banifatemi-vu-vrije-universiteit-amsterdam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


...

Epoch,Training Loss,Validation Loss
0,5.439600,4.365657


pretrained model load (from Epoch 9)...


<ipython-input-10-6deffa7b05d7>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,5.474900,4.397652


## BLiMP Evaluation for Epoch 5

In [ ]:
!cp -r "/content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline" "/content/"

In [ ]:
!cp -r "/content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/lm_eval" "/content/"

In [ ]:
!mkdir model_folder
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/config.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/merges.txt" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/model.safetensors" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/special_tokens_map.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/tokenizer.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/tokenizer_config.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/training_args.bin" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6/vocab.json" "/content/model_folder"

In [ ]:
#@title Setup script { display-mode: "form" }
#@markdown Run this cell to install the necessary packages (may take a few minutes).
%%shell
# Remove previous installation if it exists
cd /content
# mkdir model_folder
pip uninstall -y lm-eval
# rm -rf evaluation-pipeline/

# Install evaluation-pipeline
# cp -r /content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline /content/
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
# unzip filter_data.zip

Obtaining file:///content/evaluation-pipeline
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/accelerate (to revision main) to /tmp/pip-install-ong9qsxz/accelerate_9b258d1411e1480796a23ddbe4ee68ad
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-install-ong9qsxz/accelerate_9b258d1411e1480796a23ddbe4ee68ad
  Resolved https://github.com/huggingface/accelerate to commit 34c1779828b3d0769992e6492e6de93d869f71b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of lm-eval to determine which version is compatible with other requirements. This could take a while.
ERROR: Could no

CalledProcessError: Command '# Remove previous installation if it exists
cd /content
# mkdir model_folder
pip uninstall -y lm-eval
# rm -rf evaluation-pipeline/

# Install evaluation-pipeline
# cp -r /content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline /content/
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
# unzip filter_data.zip
' returned non-zero exit status 1.

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install datasets

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machin

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b6e888cbe02fea75cbe228bdabdf926cbabd47abb062cb7b1cd7b9c468b07950
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
!pip install lm_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
#@title Load model and evaluate (BLiMP) { display-mode: "form" }
model = "/content/model_folder/" #@param {"type": "string"}
model_type = "encoder" #@param ["decoder", "encoder", "encoder-decoder"]
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
%run /content/evaluation-pipeline/babylm_eval.py \
  "$model" \
  "$model_type" \
  -t "blimp"

/content/evaluation-pipeline


Generating train split: 0 examples [00:00, ? examples/s]


» Assigning unique IDs to 'blimp_from_file+null' docs


INFO:lm_eval.evaluator:
» Assigning unique IDs to 'blimp_from_file+null' docs


Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

...


» Constructing 'blimp_from_file+null' contexts and requests


INFO:lm_eval.evaluator:
» Constructing 'blimp_from_file+null' contexts and requests
100%|██████████| 5535/5535 [00:01<00:00, 4297.14it/s]


» Running all `loglikelihood` requests



INFO:lm_eval.evaluator:
» Running all `loglikelihood` requests
100%|██████████| 11070/11070 [01:17<00:00, 142.65it/s]


subject_verb_agreement:	54.72%

Scores:
anaphor_agreement:	66.72%
argument_structure:	61.93%
binding:	64.14%
control_raising:	61.20%
determiner_noun_agreement:	76.62%
ellipsis:	46.82%
filler_gap:	63.62%
irregular_forms:	83.72%
island_effects:	38.75%
npi_licensing:	48.33%
quantifiers:	73.80%
subject_verb_agreement:	54.72%


In [ ]:
!cp -r "/content/model_folder/zeroshot" "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e5_v40_l6"

## BLiMP Evaluation for Epoch 10

In [ ]:
!cp -r "/content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline" "/content/"

In [ ]:
!cp -r "/content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/lm_eval" "/content/"

In [ ]:
!mkdir model_folder
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/config.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/merges.txt" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/model.safetensors" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/special_tokens_map.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/tokenizer.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/tokenizer_config.json" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/training_args.bin" "/content/model_folder"
!cp "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6/vocab.json" "/content/model_folder"

In [ ]:
#@title Setup script { display-mode: "form" }
#@markdown Run this cell to install the necessary packages (may take a few minutes).
%%shell
# Remove previous installation if it exists
cd /content
# mkdir model_folder
pip uninstall -y lm-eval
# rm -rf evaluation-pipeline/

# Install evaluation-pipeline
# cp -r /content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline /content/
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
# unzip filter_data.zip

Obtaining file:///content/evaluation-pipeline
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/accelerate (to revision main) to /tmp/pip-install-47pw7pr1/accelerate_b7bc94645c414f0fb4e8736a81a45ef8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-install-47pw7pr1/accelerate_b7bc94645c414f0fb4e8736a81a45ef8
  Resolved https://github.com/huggingface/accelerate to commit 423fbbfdea0eda4e5b2db7bad2194da293dc69f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 80.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of lm-eval to determine which version is compatible with other requirements. This could take a while.
ERROR: Could no

CalledProcessError: Command '# Remove previous installation if it exists
cd /content
# mkdir model_folder
pip uninstall -y lm-eval
# rm -rf evaluation-pipeline/

# Install evaluation-pipeline
# cp -r /content/drive/MyDrive/VU Thesis/Code/BliMP_eval_code/evaluation-pipeline /content/
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
# unzip filter_data.zip
' returned non-zero exit status 1.

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_mach

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ffec285af0081aeafdd513449480da0f8cc9b99f1e1abc0a6792999fd203d1e9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
!pip install lm_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Load model and evaluate (BLiMP) { display-mode: "form" }
model = "/content/model_folder/" #@param {"type": "string"}
model_type = "encoder" #@param ["decoder", "encoder", "encoder-decoder"]
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
%run /content/evaluation-pipeline/babylm_eval.py \
  "$model" \
  "$model_type" \
  -t "blimp"

/content/evaluation-pipeline


Generating train split: 0 examples [00:00, ? examples/s]


» Assigning unique IDs to 'blimp_from_file+null' docs


INFO:lm_eval.evaluator:
» Assigning unique IDs to 'blimp_from_file+null' docs


Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

...


» Constructing 'blimp_from_file+null' contexts and requests


INFO:lm_eval.evaluator:
» Constructing 'blimp_from_file+null' contexts and requests
100%|██████████| 5535/5535 [00:00<00:00, 6927.98it/s]


» Running all `loglikelihood` requests



INFO:lm_eval.evaluator:
» Running all `loglikelihood` requests
100%|██████████| 11070/11070 [01:23<00:00, 132.87it/s]


subject_verb_agreement:	57.27%

Scores:
anaphor_agreement:	62.12%
argument_structure:	62.38%
binding:	62.94%
control_raising:	61.27%
determiner_noun_agreement:	76.32%
ellipsis:	49.19%
filler_gap:	63.54%
irregular_forms:	79.24%
island_effects:	40.88%
npi_licensing:	47.98%
quantifiers:	73.96%
subject_verb_agreement:	57.27%


In [ ]:
!cp -r "/content/model_folder/zeroshot" "/content/drive/MyDrive/VU Thesis/Code/baby_models/V40L6/model_e10_v40_l6"